**Import Libraries**

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img ,img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
import cv2



**Download and Generate Data**

In [3]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d ravirajsinh45/real-life-industrial-dataset-of-casting-product

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 81% 81.0M/100M [00:01<00:00, 50.0MB/s]
100% 100M/100M [00:01<00:00, 66.9MB/s] 


In [4]:
!unzip '/content/real-life-industrial-dataset-of-casting-product.zip'

Streaming output truncated to the last 5000 lines.
  inflating: casting_data/casting_data/train/def_front/cast_def_0_5259.jpeg  
  inflating: casting_data/casting_data/train/def_front/cast_def_0_526.jpeg  
  inflating: casting_data/casting_data/train/def_front/cast_def_0_5260.jpeg  
  inflating: casting_data/casting_data/train/def_front/cast_def_0_5262.jpeg  
  inflating: casting_data/casting_data/train/def_front/cast_def_0_5265.jpeg  
  inflating: casting_data/casting_data/train/def_front/cast_def_0_527.jpeg  
  inflating: casting_data/casting_data/train/def_front/cast_def_0_5270.jpeg  
  inflating: casting_data/casting_data/train/def_front/cast_def_0_5272.jpeg  
  inflating: casting_data/casting_data/train/def_front/cast_def_0_5274.jpeg  
  inflating: casting_data/casting_data/train/def_front/cast_def_0_528.jpeg  
  inflating: casting_data/casting_data/train/def_front/cast_def_0_5282.jpeg  
  inflating: casting_data/casting_data/train/def_front/cast_def_0_5283.jpeg  
  inflating: cas

In [5]:
train_generator = ImageDataGenerator(rescale=1/255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2 )
train_data = train_generator.flow_from_directory('../content/casting_data/casting_data/train',
                                          class_mode='binary',
                                          batch_size=8,
                                          target_size=(64,64),
                                          color_mode='grayscale')
test_generator = ImageDataGenerator(rescale=1/255, horizontal_flip=True, zoom_range=0.2, shear_range=0.2 )
test_data = test_generator.flow_from_directory('../content/casting_data/casting_data/test',
                                          class_mode='binary',
                                          batch_size=8,
                                          target_size=(64,64),
                                          color_mode='grayscale')

Found 6633 images belonging to 2 classes.
Found 715 images belonging to 2 classes.


**Model Definition**

In [6]:
model= Sequential()
#layer1
model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), 
                                 activation='relu', padding='same',
                                 input_shape=(64,64,1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=2))

#layer2
model.add(tf.keras.layers.Conv2D(filters=8, kernel_size=(3,3), 
                                 activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=2))
model.add(tf.keras.layers.Flatten())

#neural network
model.add(tf.keras.layers.Dense(units=128, activation= 'relu'))
model.add(tf.keras.layers.Dense(units=128, activation= 'relu'))
model.add(tf.keras.layers.Dense(units=1, activation= 'sigmoid'))

In [7]:
#configure model for training
model.compile(optimizer= 'Adam', loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.FalseNegatives()])

**Model Training**

In [8]:
#train
model.fit_generator(train_data, validation_data= test_data, epochs=12)


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/12
830/830 [==============================] - 32s 37ms/step - loss: 0.5494 - binary_accuracy: 0.7028 - false_negatives: 1222.0000 - val_loss: 0.3736 - val_binary_accuracy: 0.8308 - val_false_negatives: 46.0000
Epoch 2/12
830/830 [==============================] - 30s 37ms/step - loss: 0.3501 - binary_accuracy: 0.8409 - false_negatives: 485.0000 - val_loss: 0.2835 - val_binary_accuracy: 0.8881 - val_false_negatives: 12.0000
Epoch 3/12
830/830 [==============================] - 31s 37ms/step - loss: 0.2893 - binary_accuracy: 0.8722 - false_negatives: 360.0000 - val_loss: 0.3298 - val_binary_accuracy: 0.8531 - val_false_negatives: 61.0000
Epoch 4/12
830/830 [==============================] - 31s 37ms/step - loss: 0.2739 - binary_accuracy: 0.8826 - false_negatives: 319.0000 - val_loss: 0.1841 - val_binary_accuracy: 0.9175 - val_false_negatives: 22.0000
Epoch 5/12
830/830 [==============================] - 31s 37ms/step - loss: 0.2039 - binary_accuracy: 0.9168 - false_negatives: 220

In [9]:
model.save('top_model.h5')

In [21]:
test_image= load_img('/content/casting_512x512/casting_512x512/def_front/cast_def_0_2304.jpeg',
                     target_size=(64,64), color_mode='grayscale')

t_image=cv2.imread('/content/casting_512x512/casting_512x512/def_front/cast_def_0_2304.jpeg',0)
t_image=t_image/255


**Model Test**

In [22]:

test_image= img_to_array(test_image)
test_image= test_image/255
test_image = np.expand_dims(test_image,axis=0)
result = model.predict(test_image)

if result[0]<=0.5:
    print('OK')
else :
    print('Defective')

Defective
